### Antonio Alejo Module 5

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
game = pd.read_json(r'C:\Users\tonya\Downloads\jeopardy.json')

In [3]:
game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   category     216930 non-null  object
 1   air_date     216930 non-null  object
 2   question     216930 non-null  object
 3   value        213296 non-null  object
 4   answer       216930 non-null  object
 5   round        216930 non-null  object
 6   show_number  216930 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 11.6+ MB


In [4]:
game.head()

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680


In [5]:
game['value'].value_counts()

$400       42244
$800       31860
$200       30455
$600       20377
$1000      19539
           ...  
$5,401         1
$1,183         1
$1,203         1
$11,600        1
$11,200        1
Name: value, Length: 149, dtype: int64

Since a Jeopardy! board consists of questions at \\$200, \\$400, \\$600, \\$800, and \\$1,000, the classifier I create will consist of 'low value' and 'high value'. Those questions that are \\$200 and \\$400 will be considered 'low value'. The rest will be considered as 'high value'. Another variable will be made as a binary classifier for the Naive Bayes analysis. The analysis will be done to predict whether a question is of high or low value based on the category of the question.

In [6]:
#create a function to determine whether a question is high or low value
def high_or_low(question):
    if question['value'] == '$200' or question['value'] == '$400':
        return 'low_value'
    else:
        return 'high_value'

#add new variable to the dataset that implements the new function    
game['h_or_l'] = game.apply(high_or_low, axis=1)

#create a function similar to high_or_low that gives a 0 for low_value and 1 for high_value
def binary(value):
    if value['h_or_l'] == 'low_value':
        return 0
    else:
        return 1

#again, create a new variable that implements the new function    
game['binary'] = game.apply(binary, axis=1)

In [7]:
game.head()

,category,air_date,question,value,answer,round,show_number,h_or_l,binary
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680,low_value,0
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680,low_value,0
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680,low_value,0
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680,low_value,0
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680,low_value,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(game['category'].astype(str), game['binary'], 
                                                    test_size=0.2, random_state=1)

In [9]:
tfidf_vectorizer = TfidfVectorizer(use_idf = True)
X_train_tf = tfidf_vectorizer.fit_transform(X_train)
X_test_tf = tfidf_vectorizer.transform(X_test)

In [10]:
nb = MultinomialNB()
nb.fit(X_train_tf, y_train)

MultinomialNB()

In [11]:
predictions = nb.predict(X_test_tf)

In [12]:
accuracy = accuracy_score(y_test, predictions)
accuracy

0.6469598487991518

This model, given our variables, is about 65% accurate.

I am also curious to see the accuracy of a model that predicts whether a question is of high or low value (as I've defined it) based on the round of Jeopardy! that the question was asked during. A similar approach will be taken.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(game['round'].astype(str), game['binary'], 
                                                    test_size=0.2, random_state=1)

In [14]:
tfidf_vectorizer = TfidfVectorizer(use_idf = True)
X_train_tf = tfidf_vectorizer.fit_transform(X_train)
X_test_tf = tfidf_vectorizer.transform(X_test)

In [15]:
nb = MultinomialNB()
nb.fit(X_train_tf, y_train)

MultinomialNB()

In [16]:
predictions = nb.predict(X_test_tf)

In [17]:
accuracy = accuracy_score(y_test, predictions)
accuracy

0.6645461669663025

The accuracy of this model is similar to the previous one, at about 66%. It would be interesting to see if the model is more or less accurate if I were to change the way I define high and low value questions.